# Neural Networks

Neural networks can be constructed using the `torch.nn` package. `nn` depends on `autograd` to define models and differentiate them. An `nn.Module` contains layers, and a method `forward(input)`that returns the `output`.

Example: 

![ConvNet](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

## Define the network

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    
    # 1 input image channel, 6 output channels, 3x3 square convolutiuon

    # kernel
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3) 

    # affine operation: y = Wx + b
    self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 from image dimension
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):

    # Max pooling over a (2, 2) window
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    # If the size is a square you can only specify a single number
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    
    x = x.view(-1, self.num_flat_features(x))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

  def num_flat_features(self, x):
    size = x.size()[1:] # all dimension except the batch dimension
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

In [21]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd` 👻. You can use any of the Tensor operations in the `forward` function.

The learnable parameters of a model are returned by `net.parameters()`:

In [22]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. 

> Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [23]:
input = torch.randn(1, 1, 32, 32) # nSamples x nChannels x Height x Width
input

tensor([[[[ 0.5477,  1.9696, -1.0139,  ...,  0.0520, -0.5070, -1.5527],
          [-0.0891,  1.1150,  0.6102,  ..., -0.5599,  0.7167,  0.8280],
          [-1.0423, -0.0944,  0.4863,  ...,  1.5127,  0.8198,  0.6389],
          ...,
          [ 2.2860, -0.2799,  0.2085,  ...,  0.8444,  0.7534, -0.7493],
          [ 0.8204, -1.6026, -0.6270,  ..., -1.1169,  0.5376,  1.1022],
          [ 0.1709,  0.7838, -1.2950,  ..., -0.3746, -0.2765,  1.9111]]]])

In [24]:
out = net(input)
print(out)

tensor([[-0.0322, -0.0178,  0.0594, -0.0036, -0.0501,  0.1093,  0.0533,  0.0458,
         -0.0099, -0.1082]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [0]:
net.zero_grad()
out.backward(torch.randn(1, 10))

> Note: \
> `torch.nn` only supports **mini-batches**. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.\
For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`.\
If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.

## Loss function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different [loss functions](https://pytorch.org/docs/nn.html#loss-functions) under the nn package . A simple loss is: `nn.MSELoss` which computes the mean-squared error between the input and the target.

In [26]:
output = net(input)
target = torch.randn(10) # a dummy target for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.9301, grad_fn=<MseLossBackward>)

Now, if you follow `loss` in the backward direction, using its `.grad_fn` attribute, you will see a graph of computations that looks like this:

```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

So, when we call `loss.backward()`, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

For illustration, follow a few steps backward:

In [27]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop

To backpropagate the error all we have to do is to `loss.backward()`. You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.

In [28]:
net.zero_grad() # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward: ')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward:')
print(net.conv1.bias.grad)

conv1.bias.grad before backward: 
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward:
tensor([-0.0022,  0.0010,  0.0060, -0.0046, -0.0027,  0.0137])


## Update the weights

The simplest update rule used in practice is the **Stochastic Gradient Descent (SGD)**:

```
weight = weight - learning_rate * gradient
```



Implement this using simple Python code:

In [0]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.

In [0]:
import torch.optim as optim

# Create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in training loop:
optimizer.zero_grad() # zero the gradient buffer
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # update